# MNIST Dataset Processing and Neural Network

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler


## Step 1: Download and Process MNIST Dataset

In [ ]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Resize images to 16x16 (256 pixels)
x_train = x_train[:, ::2, ::2]  # Downsample by skipping every other pixel
x_test = x_test[:, ::2, ::2]

# Normalize pixel values to range [0.0, 1.0] (0.0 for white, 1.0 for black)
x_train = 1 - x_train / 255.0
x_test = 1 - x_test / 255.0

# Flatten images into 256-pixel vectors
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Convert targets to one-hot encoding
y_train_onehot = to_categorical(y_train, 10)
y_test_onehot = to_categorical(y_test, 10)

## Step 2: Save Processed Data to CSV Files

In [ ]:
# Save input data to input.csv
input_df = pd.DataFrame(x_train_flat)
input_df.to_csv("input.csv", index=False)

# Save target data to target.csv
target_df = pd.DataFrame(y_train_onehot)
target_df.to_csv("target.csv", index=False)

print("Saved input.csv and target.csv.")

## Step 3: Create a Neural Network using Keras

In [ ]:
import subprocess

# Define parameters for `nn_generator`
model_directory = "./trained_model"
shape_file = "./shape.txt"
input_file = "input.csv"
target_file = "target.csv"
training_verification_ratio = 0.7
learning_rate = 0.01
epochs = 100
tolerance = 0.1
num_generations = 100
log_level = 1
population_size = 4
num_offsprings = 10

# Create the command for training the neural network
command = [
    "./nn_generator",
    "--model-directory", model_directory,
    "--shape-file", shape_file,
    "--input-file", input_file,
    "--target-file", target_file,
    "--training-verification-ratio", str(training_verification_ratio),
    "--learning-rate", str(learning_rate),
    "--epochs", str(epochs),
    "--tolerance", str(tolerance),
    "--num-generations", str(num_generations),
    "--log-level", str(log_level),
    "--population-size", str(population_size),
    "--num-offsprings", str(num_offsprings)
]

# Run the command
try:
    subprocess.run(command, check=True)
    print("Model training completed successfully and saved in:", model_directory)
except subprocess.CalledProcessError as e:
    print("Error occurred during model training:", e)


## Step 4: Evaluate the Model

In [ ]:
# Evaluate on test data
loss, accuracy = model.evaluate(x_test_flat, y_test_onehot)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")